# Pegasus on Patent Dataset

This notebook contains the necessary codes to test the Pegasus model in the patent text summarization task.

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%%capture
!pip3 install git+https://github.com/huggingface/transformers
!pip3 install sentencepiece

In [32]:
DATASET_NAME = "pubmed"
DEVICE = "cuda"
CACHE_DIR = DATASET_NAME
MODEL_ID = f"google/pegasus-{DATASET_NAME}"

In [27]:
cd /content/drive/My Drive/Pesquisa/datasets/HTS

/content/drive/My Drive/Pesquisa/datasets/HTS


In [28]:
abstract = open("resumo.valid.txt").readlines()
title = open("titulo.valid.txt").readlines()

abstract = [i.replace('\n', '') for i in abstract]
title = [i.replace('\n', '') for i in title]

In [3]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

In [39]:
model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large')
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-large')

In [23]:
ARTICLE_TO_SUMMARIZE = (
 "PG&E stated it scheduled the blackouts in response to forecasts for high winds "
 "amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were "
 "scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."
)

In [35]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [36]:
def generate_answer(batch):
  inputs_dict = tokenizer(batch["abstract"], padding="max_length", max_length=4096, return_tensors="pt", truncation=True)
  inputs_dict = {k: inputs_dict[k].to(DEVICE) for k in inputs_dict}
  predicted_abstract_ids = model.generate(**inputs_dict, max_length=15, num_beams=5, length_penalty=0.8)
  batch["predicted_abstract"] = tokenizer.decode(predicted_abstract_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
  #print(batch["predicted_abstract"])
  return batch

In [37]:
batches = []
for i in range(len(abstract)):

  init = i*1
  end = init+1

  batches.append((init, end))

In [40]:
texts = []
preds = []
refs = []


f = open("candidates_pegasus.txt", 'w')


for init, end in batches:

     dataset = {'abstract': abstract[init:end], "summary": title[init:end]}
     results = generate_answer(dataset)

     texts.append(results['abstract'])
     preds.append(results['predicted_abstract'])
     refs.append(results['summary'])

     f = open("candidates_pegasus.txt", 'a')
     f.write(results['predicted_abstract'] + "\n")
     f.close()

     gc.collect()
     torch.cuda.empty_cache()

result = pd.DataFrame({'abstract':texts, "reference":refs,"candidate": preds })

TypeError: ignored